# COGS 118B Final Project
### Authors:
- Michelle Tran
- Christopher Ly

<strong style="color:red">Just for reference for working on GitHub:</strong>
<p style="color:red">Be sure to clear output for the notebook before pushing to the repo, this is to keep commit history clean. You can do this by following the sequence below:</p>

`Cell > All Output > Clear`

In [ ]:
import sys
sys.path.insert(0,'src')
import utils

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
sns.set(style='darkgrid', palette='rainbow')

## Load in data

In [ ]:
fp = 'data/telco.csv'

In [ ]:
df = pd.read_csv(fp).drop(['customerID'], axis=1)
df.head()

## Clean data

categorical_cols = df.select_dtypes(include=[object]).columns

for col in categorical_cols:
    print(col, df[col].unique())

df = utils.convert(df)

In [ ]:
df = df.drop(df[df['tenure']==0].index).reset_index(drop=True)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'])#, errors='coerce')
# df['tenure'] = df['tenure'].replace({0: 1})

# df['TotalCharges'] = df['TotalCharges'].fillna(df['MonthlyCharges'])

for col in categorical_cols:
    print(col, df[col].unique())

## Exploratory Data Analysis

In [ ]:
df = utils.revert(df)

In [ ]:
# ref: https://stackoverflow.com/questions/33179122/seaborn-countplot-with-frequencies
ax = sns.countplot(x=df['Churn'])
plt.title('Churn rate/occurrence')
ax.set_ylabel('Count')
ax.set_ylim(0, len(df))

for p in ax.patches:
    x=p.get_bbox().get_points()[:,0]
    y=p.get_bbox().get_points()[1,1]
    ax.annotate('{:.2f}%'.format(100.*y/len(df)), (x.mean(), y), 
            ha='center', va='bottom') # set the alignment of the text
    
plt.show()

In [ ]:
utils.plot_cat(df);

In [ ]:
sns.distplot(df['tenure'][df['Churn'] == 'Yes'])
sns.distplot(df['tenure'][df['Churn'] == 'No'])
plt.title('Density of Tenure in Months for Churn and No Churn')
plt.legend(['Churn', 'No Churn'])
plt.xlabel('Tenure (Months)')
plt.ylabel('Probability Density')
plt.show()

In [ ]:
sns.distplot(df['MonthlyCharges'][df['Churn'] == 'Yes'])
sns.distplot(df['MonthlyCharges'][df['Churn'] == 'No'])
plt.title('Density of Monthly Charges for Churn and No Churn')
plt.legend(['Churn', 'No Churn'])
plt.xlabel('Monthly Charges')
plt.ylabel('Probability Density')
plt.show()

In [ ]:
sns.distplot(df['TotalCharges'][df['Churn'] == 'Yes'])
sns.distplot(df['TotalCharges'][df['Churn'] == 'No'])
plt.title('Density of Total Charges for Churn and No Churn')
plt.legend(['Churn', 'No Churn'])
plt.xlabel('Total Charges')
plt.ylabel('Probability Density')
plt.show()

In [ ]:
df = utils.convert(df)

In [ ]:
scaler = StandardScaler()
ohe = OneHotEncoder(handle_unknown='ignore')

In [ ]:
df_categorical = df.select_dtypes(include=object)
df_categorical.head(2)

In [ ]:
df_numerical = df.select_dtypes(include=np.number).drop(['Churn', 'TotalCharges'], axis=1)
df_numerical.head(2)

In [ ]:
X = pd.DataFrame(ohe.fit_transform(df_categorical).todense(), columns=ohe.get_feature_names(df_categorical.columns))

In [ ]:
X = pd.concat([df_numerical,X], axis=1)
X.head(2)

In [ ]:
y = df['Churn']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# ref: https://stackoverflow.com/questions/29432629/plot-correlation-matrix-using-pandas
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(3)

## Regression Model

In [ ]:
mdl = LogisticRegression(solver='liblinear')
result = mdl.fit(X_train, y_train)

In [ ]:
score = mdl.score(X_test, y_test)
print('Accuracy Score %.2f' %(score*100))

In [ ]:
param_grid = {'C': [0.001,0.005,0.01,0.05,0.1,0.5,1,10], 
              'penalty' : ['l1', 'l2'],
              'tol' : [1e-4],
              'max_iter' : [100,500,1000]}

In [ ]:
gs = GridSearchCV(mdl, param_grid, n_jobs=-1, cv=10)
gs.fit(X_train, y_train)

In [ ]:
gs.best_params_

In [ ]:
gs.best_score_

In [ ]:
print('Accuracy Score %.2f' %(mdl.set_params(**gs.best_params_).score(X_test, y_test)*100))

In [ ]:
to_sort = {k:v for k,v in zip(X.columns.values,mdl.coef_.reshape((-1,)))}
sort = {k: v for k, v in sorted(to_sort.items(), key=lambda item: item[1])}

In [ ]:
plt.figure(figsize=(12,12))
plt.suptitle("Weights of logistic regression model")
sns.barplot(x=[x for _,x in sort.items()], y=[y for y,_ in sort.items()])

## SVM Model

In [ ]:
mdl2 = SVC(gamma='auto')
result2 = mdl2.fit(X_train, y_train)

In [ ]:
score2 = result2.score(X_test, y_test)
print('Accuracy Score %.2f' %(score2*100))

In [ ]:
param_grid2 = {'C': [0.001,0.005,0.01,0.05,0.1,0.5,1,10], 
               'kernel' : ['linear', 'poly', 'rbf', 'sigmoid'],
               'gamma': ['auto', 'scale'],
               'max_iter' : [-1,100,500,1000]}

In [ ]:
gs2 = GridSearchCV(result2, param_grid2, n_jobs=-1, cv=10)
gs2.fit(X_train, y_train)

In [ ]:
gs2.best_params_

In [ ]:
gs2.best_score_

In [ ]:
print('Accuracy Score %.2f' %(mdl2.set_params(**gs2.best_params_).score(X_test, y_test)*100))

## AdaBoost Model

In [ ]:
mdl3 = AdaBoostClassifier()
result3 = mdl3.fit(X_train, y_train)

In [ ]:
score3 = result3.score(X_test, y_test)
print('Accuracy Score %.2f' %(score3*100))

In [ ]:
param_grid3 = {'n_estimators': [10,25,50,100,250], 
               'learning_rate' : [0.01,0.05,0.1,0.5,1,10]}

In [ ]:
gs3 = GridSearchCV(result3, param_grid3, n_jobs=-1, cv=10)
gs3.fit(X_train, y_train)

In [ ]:
gs3.best_params_

In [ ]:
gs3.best_score_

In [ ]:
print('Accuracy Score %.2f' %(mdl3.set_params(**gs3.best_params_).score(X_test, y_test)*100))

In [ ]:
to_sort = {k:v for k,v in zip(X.columns.values,mdl3.feature_importances_)}
sort = {k: v for k, v in sorted(to_sort.items(), key=lambda item: item[1])}

In [ ]:
plt.figure(figsize=(12,12))
plt.suptitle("Weights of AdaBoost model")
sns.barplot(x=[x for _,x in sort.items()], y=[y for y,_ in sort.items()])

In [ ]:
to_sort = {k:v for k,v in zip(X.columns.values,mdl3.feature_importances_)}
sort = {k: v for k, v in sorted(to_sort.items(), key=lambda item: item[1]) if v!=0}

In [ ]:
plt.figure(figsize=(12,8))
plt.suptitle("Weights of AdaBoost model")
sns.barplot(x=[x for _,x in sort.items()], y=[y for y,_ in sort.items()])

## ANN Model (if time permits)

## Discussion